<a href="https://colab.research.google.com/github/Theanh130124/Hands_on_LLM/blob/master/refine_medical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#DOWNLOAD DATASET


In [1]:
!pip install huggingface_hub datasets pandas

import pandas as pd
from huggingface_hub import snapshot_download

# Tải dataset
snapshot_download(
    repo_id="pythera/medical-o1-reasoning-SFT-vi",
    local_dir="./medical-o1-reasoning-SFT-vi",
    repo_type="dataset"
)

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/86.1M [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/2.46k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/484 [00:00<?, ?B/s]

Dataset đã được tải xuống!


#LOAD DATASET

In [5]:
df = pd.read_parquet('./medical-o1-reasoning-SFT-vi/data/train-00000-of-00001.parquet')
print(f"Dataset shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
for col in df.columns:
    sample_value = df[col].iloc[0] if len(df) > 0 else "None"
    print(f"{col}: {str(sample_value)[:100]}...")

Đang đọc dataset...
Dataset shape: (25371, 6)
Columns: ['Question', 'Complex_CoT', 'Response', 'Question_Vi', 'Response_Vi', 'Complex_CoT_Vi']

=== CẤU TRÚC DATASET ===
Question: A 61-year-old woman with a long history of involuntary urine loss during activities like coughing or...
Complex_CoT: Okay, let's think about this step by step. There's a 61-year-old woman here who's been dealing with ...
Response: Cystometry in this case of stress urinary incontinence would most likely reveal a normal post-void r...
Question_Vi: Phụ nữ 61 tuổi với một lịch sử lâu dài của việc mất nước tiểu không tự nguyện trong các hoạt động nh...
Response_Vi: Trong trường hợp này, xét nghiệm Cystometry của bệnh suyễn mao quản sẽ cho thấy một lượng dư thừa bì...
Complex_CoT_Vi: Okay, hãy nghĩ về bước này một cách có hệ thống. Có một phụ nữ 61 tuổi ở đây đã phải đối mặt với vấn...


# Load model & tokenizer

In [17]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

model_name = "VietAI/envit5-translation"
tokenizer = AutoTokenizer.from_pretrained(model_name)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model.to(device)



T5ForConditionalGeneration(
  (shared): Embedding(50048, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(50048, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [26]:
PROMPT = (
    "Hãy dịch đoạn văn tiếng Anh dưới đây sang tiếng Việt:\n"
    "Văn bản tiếng Anh:\n\"\"\"\n{input}\n\"\"\"\n"
    "Chỉ trả về bản dịch tiếng Việt, không thêm chú thích hay giải thích:"
)


#REFINE FUNC

In [27]:
from tqdm import tqdm

def refine_texts(texts, max_length=256, batch_size=8, num_beams=4, temperature=0.2):
    results = []
    model.eval()
    for i in tqdm(range(0, len(texts), batch_size)):
        batch = texts[i:i+batch_size]
        prompts = [PROMPT.format(input=t) for t in batch]
        inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True, max_length=1024).to(device)

        with torch.no_grad():
            out = model.generate(
                **inputs,
                max_new_tokens=max_length,
                num_beams=num_beams,
                temperature=temperature,
                early_stopping=True,
                no_repeat_ngram_size=3
            )
        decoded = tokenizer.batch_decode(out, skip_special_tokens=True)
        decoded = [d.strip() for d in decoded]
        results.extend(decoded)
    return results

In [28]:
sample_size = 3

cols_to_refine_english = ["Question", "Response", "Complex_CoT"]
cols_vietnamese_output = ["Question_Vi", "Response_Vi", "Complex_CoT_Vi"]

texts_by_col = {}
for col in cols_to_refine_english:
    if col in df.columns:
        texts_by_col[col] = [x if x is not None else "" for x in df[col].head(sample_size)]
        print(f"Column '{col}': lấy {len(texts_by_col[col])} mẫu TIẾNG ANH để dịch sang Việt")
    else:
        print(f"WARNING: column '{col}' not in dataset")

# Chạy dịch từ Anh sang Việt
refined_by_col = {}
for col, texts in texts_by_col.items():
    print(f"\nDịch column: {col} (#rows = {len(texts)}) - ANH → VIỆT")
    refined_by_col[col] = refine_texts(texts, batch_size=2)


print("\n" + "="*60)
print("KẾT QUẢ DỊCH TỪ ANH SANG VIỆT - SO SÁNH VỚI BẢN DỊCH CŨ")
print("="*60)

for i, col_english in enumerate(refined_by_col.keys()):
    col_vietnamese = cols_vietnamese_output[i] if i < len(cols_vietnamese_output) else f"{col_english}_Vi"

    print(f"\n--- {col_english} → {col_vietnamese} ---")
    for j in range(min(3, len(texts_by_col[col_english]))):
        print(f"\nMẫu {j+1}:")
        print(f"ANH GỐC:    {str(texts_by_col[col_english][j])}...")
        print(f"VIỆT MỚI:   {str(refined_by_col[col_english][j])}...")

        if col_vietnamese in df.columns:
            viet_old = df[col_vietnamese].iloc[j] if j < len(df) else "N/A"
            print(f"VIỆT CŨ:   {str(viet_old)}...")

        print("-" * 60)

Column 'Question': lấy 3 mẫu TIẾNG ANH để dịch sang Việt
Column 'Response': lấy 3 mẫu TIẾNG ANH để dịch sang Việt
Column 'Complex_CoT': lấy 3 mẫu TIẾNG ANH để dịch sang Việt

Dịch column: Question (#rows = 3) - ANH → VIỆT


100%|██████████| 2/2 [00:04<00:00,  2.29s/it]



Dịch column: Response (#rows = 3) - ANH → VIỆT


100%|██████████| 2/2 [00:10<00:00,  5.30s/it]



Dịch column: Complex_CoT (#rows = 3) - ANH → VIỆT


100%|██████████| 2/2 [00:13<00:00,  6.99s/it]


KẾT QUẢ DỊCH TỪ ANH SANG VIỆT - SO SÁNH VỚI BẢN DỊCH CŨ

--- Question → Question_Vi ---

Mẫu 1:
ANH GỐC:    A 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings, what would cystometry most likely reveal about her residual volume and detrusor contractions?...
VIỆT MỚI:   en: "" A 61-year-old woman with a long history of involuntary loss of urine during activities such as ho or hắt hơi but no leakage at night undergoes a gynecological examination and Q - tip test. Based on these findings, what would cystometry most likely reveal about her residual volume and detrusor contractions? """ Only return the Vietnamese translation, without any additional captions or explanations:...
VIỆT CŨ:   Phụ nữ 61 tuổi với một lịch sử lâu dài của việc mất nước tiểu không tự nguyện trong các hoạt động như ho hoặc hắt hơi nhưng không có rò rỉ ban đêm trải qu